In [ ]:
import numpy as np
import pysindy as ps
from pysindy.feature_library import PolynomialLibrary, FourierLibrary, CustomLibrary, ConcatLibrary
import matplotlib.pyplot as plt

from generate_data import generate_population_data

In [ ]:
t = np.linspace(0, 100, 1001)

x0 = [50, 50]
x0_val = [100, 30]

u = lambda t : np.sin(t)
u_val = lambda t : np.cos(2*t)

x, x_dot = generate_population_data(t, x0, u)
x_val, _ = generate_population_data(t, x0_val, u_val)

In [ ]:
model = ps.SINDy(
    feature_library=PolynomialLibrary(degree=2), # 2, 1
    # feature_library=FourierLibrary(n_frequencies=1), # 2, 1
    optimizer=ps.STLSQ(threshold=0.001), # 0.001, 0.01, 0.1
    feature_names=[f'x{i+1}' for i in range(len(x_dot[0]))]+['u']
    )
model.fit(x=x, x_dot=x_dot, t=t, u=u(t))
model.print()

In [ ]:
for x_num in range(len(x0)):
    q = 'Biblioteka funkcji & Próg & $\Dot{x}'
    print(f'{q}_{x_num+1}$ & $E_{x_num+1}$ \\\\')
    for i, library in enumerate([FourierLibrary(n_frequencies=2), FourierLibrary(n_frequencies=1), PolynomialLibrary(degree=2), PolynomialLibrary(degree=1)]):
        print(f'\\hline')
        for threshold in range(3):
            threshold = 10**(threshold-3) if i >= 2 else 100*10**(threshold-3)
            name = ['Trygonometryczna (st. 2)', 'Trygonometryczna (st. 1)', 'Wielomiany (st. 2)', 'Liniowa']
            model = ps.SINDy(
                feature_library=library,
                optimizer=ps.STLSQ(threshold=threshold),
                feature_names=[f'x{i+1}' for i in range(len(x0))]+['u'])
            model.fit(x=x, x_dot=x_dot, t=t, u=u(t))
            x_sim = model.simulate(x0=x0_val, t=t, u=u_val)
            mse = ((x_sim - x_val)**2).mean(axis=0)
            if len(model.equations()[x_num].split('+')) > 2:
                eq = model.equations()[x_num].split('+')[0] + '+' + model.equations()[x_num].split('+')[1] + '\dots'
            else:
                eq = model.equations()[x_num]
            eq = eq.replace('.', ',').replace('+ -', '- ').replace('sin', '\sin').replace('cos', '\cos').replace('(1 x1)', '(x_1)').replace('(1 x2)', '(x_2)').replace('x2', 'x_2').replace('x1', 'x_1').replace('(1 u)', '(u)')
            print(f"{name[i]} & {(str(threshold).replace('.', ',') + ' &').replace(',0 &', ' &')} ${eq}$ & {str(round(mse[x_num], 2)).replace('.', ',')} \\\\")
    print('\n\n')

In [ ]:
x_sim = model.simulate(x0=x0_val, t=t, u=u_val)
mse = ((x_sim - x_val)**2).mean(axis=0)
print(f'Błąd średniokwadratowy x1: {mse[0]:.2f}, x2: {mse[1]:.2f}')

In [ ]:
plt.plot(t, x_val[:, 0])
plt.plot(t, x_val[:, 1])
plt.plot(t, x_sim[:, 0], "r--")
plt.plot(t, x_sim[:, 1], "k--")
plt.ylim(0, max(x_val[:, 0]*1.4))
plt.xlim(0, max(t))
plt.grid()
plt.legend(["Populacja ofiar", "Populacja drapieżników", "Symulacja modelu ofiar", "Symulacja modelu drapieżników"])
plt.xlabel("Czas [dni]")
ax1 = plt.gca()
ax2 = ax1.twinx()
ax2.spines['right'].set_color('green')
ax2.yaxis.label.set_color('green')
ax2.tick_params(axis='y', colors='green')
ax2.plot(t, u_val(t), 'g--', alpha=0.4)
ax2.set_ylabel("Sterowanie")
ax2.set_ylim(-4, 4)
ax1.set_ylabel("Liczba osobników")
ax2.set_ylabel("Sterowanie")
plt.show()